In [2]:
!pip install dicom2nifti
!pip install monai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 938 kB/s 
     |████████████████████████████████| 2.0 MB 9.0 MB/s 
     |████████████████████████████████| 12.8 MB 22.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 990 kB 5.1 MB/s 


In [3]:
from glob import glob
import shutil
import os
import dicom2nifti
import nibabel as nib
import numpy as np
# from tqdm import tqdm

from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.utils import first
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism
from matplotlib import pyplot as plt

In [4]:
Number_slices = 65;

#Function to create groups of 65 dcom files to divide every paitent with unique no.
def create_group(input_path,output_path):
    for patient in glob(input_path + '/*'):
            patient_name = os.path.basename(os.path.normpath(patient))

            # Here we need to calculate the number of folders which mean into how many groups we will divide the number of slices
            number_folders = int(len(glob(patient + '/*')) / Number_slices)

            for i in range(number_folders):
                output_path_name = os.path.join(output_path, patient_name + '_' + str(i))
                os.mkdir(output_path_name)

                # Move the slices into a specific folder so that you will save memory in your desk
                for i, file in enumerate(glob(patient + '/*')):
                    if i == Number_slices + 1:
                        break

                    shutil.move(file, output_path_name)

In [5]:
#Grouping labels into bundle of 65.

input_path = '/content/drive/MyDrive/Dataset/decom_files/labels'
output_path = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainSegmentation'

create_group(input_path, output_path)

#grouping images into bundle of 65.

input_path_images = '/content/drive/MyDrive/Dataset/decom_files/images'
output_path_images = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainVolumes'

create_group(input_path_images,output_path_images)

In [6]:
#Converting groups of decoms files into Nifti files. 
def dcm2nifti(in_dir, out_dir):
    '''
    This function will be used to convert dicoms into nifti files after creating the groups with 
    the number of slices that you want.
    `in_dir`: the path to the folder where you have all the patients (folder of all the groups).
    `out_dir`: the path to the output, which means where you want to save the converted nifties.
    '''
    list = glob(in_dir)
    for folder in list:
        patient_name = os.path.basename(os.path.normpath(folder))
        dicom2nifti.dicom_series_to_nifti(folder, os.path.join(out_dir, patient_name + '.nii.gz'))


In [7]:
dicom2nifti.settings.disable_validate_slice_increment()

In [12]:
in_path_img = '/content/drive/MyDrive/Dataset/decom_groups/images/*'
in_path_labels = '/content/drive/MyDrive/Dataset/decom_groups/labels/*'

out_path_img = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainVolumes'
out_path_labels = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainSegmentation'

# dcm2nifti(in_path_img,out_path_img) #for images
dcm2nifti(in_path_labels,out_path_labels)

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#Function to find if their an empty files in the dataset.


def find_empy(in_dir):
    '''
    This function will help you to find the empty volumes that you may not need for your training
    so instead of opening all the files and search for the empty ones, them use this function to make it quick.
    '''
    
    list_patients = []
    for patient in glob(os.path.join(in_dir, '*')):
        img = nib.load(patient)

        if len(np.unique(img.get_fdata())) ==1 :
            print(os.path.basename(os.path.normpath(patient)))
            list_patients.append(os.path.basename(os.path.normpath(patient)))
    
    return list_patients

In [13]:
in_path = '/content/drive/MyDrive/Dataset/Data_Train_Test/TrainSegmentation/'
list_paitents = find_empy(in_path)
# list_paitents

liver_51_0.nii.gz
liver_3_0.nii.gz
liver_3_1.nii.gz
liver_2_0.nii.gz
liver_2_1.nii.gz
liver_2_2.nii.gz
